In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = 'key_for_csrf_token_required_for_form_data'

db_path = os.path.abspath("synthetic_data/teleCC.sqlite")
app.config['SQLALCHEMY_DATABASE_URI'] = f"sqlite:///{db_path}"

# Track modifications is often unnecessary and can be disabled
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

In [7]:
type(db)

flask_sqlalchemy.extension.SQLAlchemy

In [2]:
from sqlalchemy import inspect

with app.app_context():
    inspector = inspect(db.engine)

    for table in inspector.get_table_names():
        print(table)


patient_teleCC
patient_teleCC_history
patient_teleCC_state


In [5]:
with app.app_context():
    db.session.close()

In [35]:
from sqlalchemy import text
with app.app_context():
    results = db.session.execute(text("select count(*) as n from patient_teleCC"))

results.fetchone()[0]

19838

In [37]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base

# Automap base setup
Base = automap_base()

# Reflect the tables from the database using autoload_with
with app.app_context():
    Base.prepare(autoload_with=db.engine)

    # List available tables after reflection
    if len(Base.classes) == 0:
        print("No tables were found during reflection.")
    else:
        print("Available mapped tables:")
        for table_name in Base.classes.keys():
            print(table_name)

        # Access the patient_teleCC table
        PatientTeleCC = Base.classes.patient_teleCC

        # Query the table using the ORM
        with db.session.begin():
            results = db.session.query(PatientTeleCC).limit(5).all()
            for row in results:
                print(row.__dict__)  # Print each row as a dictionary


No tables were found during reflection.


In [28]:
len(Base.classes)

0